In [1]:
## Graphdb configuration
NEO4J_URI="neo4j+s://8a886660.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="9FYDkCTM2Vq4qxWFFwik0uYP6BJ-fReP9XOYj-oDqZ4"

import os
os.environ["NEO4J_URI"]=NEO4J_URI
os.environ["NEO4J_USERNAME"]=NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"]=NEO4J_PASSWORD

from langchain_community.graphs import Neo4jGraph
neo4j_graph=Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
)

/tmp/ipykernel_5698/919926656.py:12: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  neo4j_graph=Neo4jGraph(


In [2]:
# Cypher query to count nodes and relationships
check_nodes_query = "MATCH (n) RETURN count(n) AS node_count"
check_relationships_query = "MATCH ()-[r]->() RETURN count(r) AS relationship_count"

# Execute the queries
node_count = neo4j_graph.query(check_nodes_query)
relationship_count = neo4j_graph.query(check_relationships_query)

node_count, relationship_count

([{'node_count': 2487}], [{'relationship_count': 9003}])

In [3]:
import pandas as pd

# Define the bucket and file names
bucket_name = 'mimicivliza'  # Replace with your bucket name
mimic_iv_bhc = f's3://{bucket_name}/sample_data_100.csv'

# Load the files
mimic_iv_bhc_100 = pd.read_csv(mimic_iv_bhc)

# Display the data
mimic_iv_bhc_100.head(20)

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/fsspec/registry.py:279: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)


,note_id,input,target,input_tokens,target_tokens
0,16002318-DS-17,<SEX> F <SERVICE> SURGERY <ALLERGIES> Iodine /...,This is a ___ yo F admitted to the hospital af...,1195,75
1,15638884-DS-4,<SEX> M <SERVICE> MEDICINE <ALLERGIES> Augment...,Mr. ___ is a ___ yo man with CAD with prior MI...,3496,1143
2,12435705-DS-14,<SEX> M <SERVICE> MEDICINE <ALLERGIES> ibuprof...,Mr. ___ is a ___ w/ Ph+ve ALL on dasatanib and...,5591,1098
3,12413577-DS-4,<SEX> F <SERVICE> OBSTETRICS/GYNECOLOGY <ALLER...,"On ___, Ms. ___ was admitted to the gynecology...",1119,221
4,17967161-DS-29,<SEX> M <SERVICE> SURGERY <ALLERGIES> lisinopr...,Mr. ___ underwent an angiogram on ___ which sh...,3307,439
5,16956007-DS-20,<SEX> M <SERVICE> SURGERY <ALLERGIES> Codeine ...,Mr. ___ is a ___ who underwent an exploratory ...,4168,1209
6,16919911-DS-15,<SEX> F <SERVICE> MEDICINE <ALLERGIES> Penicil...,This is a ___ year old female with a recent di...,2059,208
7,15682570-DS-25,<SEX> M <SERVICE> MEDICINE <ALLERGIES> No Know...,"___ w/ h/o CAD ___ CABG LIMA to LAD, SVG to D1...",2215,451
8,12135369-DS-24,<SEX> F <SERVICE> MEDICINE <ALLERGIES> Compazi...,Ms ___ is a ___ year old woman with a history ...,2132,416
9,11906321-DS-20,<SEX> M <SERVICE> NEUROSURGERY <ALLERGIES> Pat...,The patient was admitted to the neurosurgery s...,2347,316


In [4]:
mimic_iv_bhc_100.nunique()

note_id          100
input            100
target           100
input_tokens     100
target_tokens     91
dtype: int64

In [6]:
rag_data = mimic_iv_bhc_100[['note_id', 'input']]
rag_data.head(4)

,note_id,input
0,16002318-DS-17,<SEX> F <SERVICE> SURGERY <ALLERGIES> Iodine /...
1,15638884-DS-4,<SEX> M <SERVICE> MEDICINE <ALLERGIES> Augment...
2,12435705-DS-14,<SEX> M <SERVICE> MEDICINE <ALLERGIES> ibuprof...
3,12413577-DS-4,<SEX> F <SERVICE> OBSTETRICS/GYNECOLOGY <ALLER...


In [7]:
rag_data['input'].iloc[0]

'<SEX> F <SERVICE> SURGERY <ALLERGIES> Iodine / Thallium-201 / Blue Dye / Iodine-Iodine Containing <ATTENDING> ___. <CHIEF COMPLAINT> Morbid obesity, BMI of 51 <MAJOR SURGICAL OR INVASIVE PROCEDURE> lap gastric bypass <HISTORY OF PRESENT ILLNESS> The patient is a ___ woman with history of obesity, multiple medical problems with a history of 7 pound weight loss and regain. Comorbid conditions include sleep apnea, hypothyroidism, back pain, iron deficiency anemia and headaches. The patient has significant allergies particularly to the blue dye and iodine. The patient was evaluated at ___ ___ ___ Program deemed a good candidate for surgical weight loss. She understands the risks, benefits and alternatives of weight loss surgery. She agrees to diet, exercise, support group and lifelong medical follow-up particularly for B12, calcium and folate levels. <PAST MEDICAL HISTORY> Past medical history includes sleep apnea, hypothyroidism, back pain, urticaria for which she is on chronic steroids,